In [1]:
import h5py
import numpy as np
import tensorflow as tf
import pathlib
import pickle
from sklearn.preprocessing import OneHotEncoder
import sys
from tqdm import tqdm

In [2]:


import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
attack_vector='cache' #network
machine_setting= 'static_browser_version' # 'dynamic_browser_version'
n_dataset_features = 15000 if attack_vector == 'cache' else 3000
max_val=8 if attack_vector == 'cache' else 8192
root_path=f'/content/drive/MyDrive/Colab Notebooks/concept-drift-online'
data_path=f'{root_path}/data/{machine_setting}/{attack_vector}'
model_0_train_path=f'2021_08_27/train.h5'


In [11]:
data_path

'/content/drive/MyDrive/Colab Notebooks/concept-drift-online/data/static_browser_version/cache'

In [12]:
sys.path.append(str(root_path)) # Add the directory to the path
from utils import cnn_lstm,small_cnn_lstm,prepare_hdf5_data,load_hdf5_data,create_subset_encoder

In [13]:
# Example usage:
hdf5_train_path = (pathlib.Path(data_path)/model_0_train_path).as_posix()

nn_config_cache={
    'features_per_layer': 256,
    'strides': 3,
    'pool_size': 4,
    'units': 32,
    'dropout': 0.2,
}

nn_config_network={
            'features_per_layer': 256,
            'pool_size': 3,
            'strides': 3,
            'units': 128,
            'dropout': 0.8,
            'kernel_size': 32,
}
num_of_classes=100
network_config=nn_config_cache if attack_vector == 'cache' else nn_config_network
model_func=cnn_lstm if attack_vector == 'cache' else small_cnn_lstm

new_model=model_func(n_dataset_features=n_dataset_features, num_of_classes=num_of_classes, **network_config)
le=OneHotEncoder()

with h5py.File(hdf5_train_path, 'r') as hdf:
    le.fit( sorted(hdf['site_name'][:].reshape(-1,1)) )

le_dir=f'{data_path}/models/label_encoder_100.pkl'
with open(le_dir, 'wb') as f:
    pickle.dump(le, f)


In [14]:
batch=256
max_epochs=100

In [15]:
# First load data using load_hdf5_data
X_data, y_data = load_hdf5_data(hdf5_train_path, le, n_samples=-1)

# Then prepare datasets using prepare_hdf5_data with the loaded data
datasets = prepare_hdf5_data(X_data, y_data, batch_size=batch, sample_validation=0.1,
                            verbose=0, max_val=max_val)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

verbose=1
new_model.fit(datasets['data'],
            validation_data=datasets['val'] ,
            epochs=max_epochs,
            callbacks=[early_stopping],
            verbose=verbose
        )


In [ ]:
model_path=f'{data_path}/models/cnn_lstm.keras'
new_model.save(model_path)

In [ ]:
new_model.summary()

In [16]:
n_open_world_labels=30
num_of_ow_classes=num_of_classes-n_open_world_labels
X_data, y_data = load_hdf5_data(hdf5_train_path, le, n_samples=-1)
cw_mask=y_data[:,-n_open_world_labels:].any(axis=1)==0
owle=create_subset_encoder(le,num_of_ow_classes)

In [17]:
datasets_cw_part = prepare_hdf5_data(X_data[cw_mask], y_data[cw_mask][:,:num_of_ow_classes], batch_size=batch,
                              sample_validation=0.1, verbose=0)

In [18]:
# Then prepare datasets using prepare_hdf5_data with the loaded data


ow_model=model_func(n_dataset_features=n_dataset_features, num_of_classes=num_of_ow_classes, **network_config)

In [19]:
ow_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 15000, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4995, 256)      │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1249, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 417, 256)       │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 105, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 105, 32)        │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3360)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 3360)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 70)             │       235,270 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 801,158 (3.06 MB)

 Trainable params: 801,158 (3.06 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

verbose=1
ow_model.fit(datasets_cw_part['data'],
            validation_data=datasets_cw_part['val'] ,
            epochs=max_epochs,
            callbacks=[early_stopping],
            verbose=verbose
        )

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 25s 547ms/step - accuracy: 0.2252 - loss: 3.3936 - val_accuracy: 0.7429 - val_loss: 1.1045
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 194ms/step - accuracy: 0.7824 - loss: 0.8985 - val_accuracy: 0.8571 - val_loss: 0.6016
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 0.9027 - loss: 0.4343 - val_accuracy: 0.8921 - val_loss: 0.4292
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.9356 - loss: 0.2659 - val_accuracy: 0.9127 - val_loss: 0.3418
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.9511 - loss: 0.2120 - val_accuracy: 0.9032 - val_loss: 0.3406
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 198ms/step - accuracy: 0.9547 - loss: 0.1723 - val_accuracy: 0.9127 - val_loss: 0.2985
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.9578 - loss: 0.1440 - val_accuracy: 0.9159 - val_loss: 0.2962
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 210ms/step - accuracy: 0.9655 - loss: 0.1269 - val_ac

In [21]:
ow_model_path=f'{data_path}/models/cnn_lstm_ow.keras'
ow_model.save(ow_model_path)

In [22]:
ow_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 15000, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4995, 256)      │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1249, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 417, 256)       │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 105, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 105, 32)        │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3360)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 3360)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 70)             │       235,270 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,403,476 (9.17 MB)

 Trainable params: 801,158 (3.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,602,318 (6.11 MB)

In [ ]:
ow_model_path

'/content/drive/MyDrive/Colab Notebooks/concept-drift-online/data/static_browser_version/network/models/cnn_lstm_ow.keras'